In [1]:
import collections
import json
import uuid

In [2]:
with open("pikov.json") as f:
    core = json.load(f)

In [15]:
guid_map = core["guidMap"]
guid_map.keys()

dict_keys(['169a81aefca74e92b45e3fa03c7021df', 'aba6ac79fd3d409da860a77c90942852', '8b057dd7a9c84b7180cb2d8d6015b833', 'c6b725e0c8bb419ca9408eec5febbde8', 'a2be632d888143e89ddfd4b1b8c8492d', '8fe06c0750344322bd773f56bdd73b0a', '8ea9b7bc02e748ae9dc2e169590e71e3', 'ff7abe687a764f9d80d67a70424c329a', '6ecf1345ea0b4865b92569971b100b09', '4e09f9df1fdf4eb4964ff9ed1b375dbb', 'bc7e9e34c3464292ba39c2b1b9dc8902', '0de9d2d0679945ca9e6957f049cc982c', '995607dcd31e477994333565511c1de2', 'c5f33d38b1104896ba54d09dba3d0acf', 'bfa8113cb5e5436ebd76ab5418b7efd1', '825142afc2934fbcb5126e149ac5ba31', '66b25e276cfb4d83a7032baaa4369b6f', '667901a1e1c54035b7e586a05fffed2e', '1f22dbb5504344fb93f57f0f0eb0ba6f', '68b5d34bb7954ad6850cd55cbae05ccf'])

In [4]:
#remapped = {}
#for key, edges in core["guidMap"].items():
#    node = {}
#    remapped[key] = node
#    for edge in edges:
#        node[edge["label"]["guid"]] = edge["to"]
#remapped

In [5]:
#o = {"guidMap": remapped}
# print(json.dumps(o, indent=2))

In [16]:
names = collections.defaultdict(list)
for key, edges in guid_map.items():
    for edge, value in edges.items():
        if edge == "169a81aefca74e92b45e3fa03c7021df":
            names[value["string"]].append(key)

In [7]:
names

defaultdict(list,
            {'name': ['169a81aefca74e92b45e3fa03c7021df'],
             'ctor': ['aba6ac79fd3d409da860a77c90942852'],
             'frame': ['8b057dd7a9c84b7180cb2d8d6015b833'],
             'frame.bitmap': ['c6b725e0c8bb419ca9408eec5febbde8'],
             'frame.duration_microseconds': ['a2be632d888143e89ddfd4b1b8c8492d'],
             'transition': ['8fe06c0750344322bd773f56bdd73b0a'],
             'transition.source': ['8ea9b7bc02e748ae9dc2e169590e71e3'],
             'transition.target': ['ff7abe687a764f9d80d67a70424c329a'],
             'resource': ['6ecf1345ea0b4865b92569971b100b09'],
             'resource.relative_path': ['4e09f9df1fdf4eb4964ff9ed1b375dbb'],
             'bitmap': ['bc7e9e34c3464292ba39c2b1b9dc8902'],
             'bitmap.resource': ['0de9d2d0679945ca9e6957f049cc982c'],
             'bitmap.rectangle': ['995607dcd31e477994333565511c1de2'],
             'rectangle': ['c5f33d38b1104896ba54d09dba3d0acf'],
             'coordinate': ['bfa8113cb5e

In [33]:
class SemanticGraphNode(object):
    def __init__(self, ctor, guid_map, guid=None):
        if guid is None:
            guid = uuid.uuid4().hex
        self._guid = guid
        self._guid_map = guid_map

        if guid in self._guid_map:
            self._edges = self._guid_map[guid]
        else:
            self._edges = {
                names["ctor"][0]: ctor,
            }
            self._guid_map[guid] = self._edges
    
    def __repr__(self):
        return json.dumps(self._edges)


class AbstractSemanticGraphProperty(object):
    def __init__(self, label_guid):
        self._label_guid = label_guid

    def __get__(self, obj, type=None):
        return self.from_json(obj, obj._edges.get(self._label_guid, {}))

    def __set__(self, obj, value):
        obj._edges[self._label_guid] = self.to_json(value)


class GuidProperty(AbstractSemanticGraphProperty):
    def __init__(self, label_guid, cls):
        super().__init__(label_guid)
        self._cls = cls

    def from_json(self, obj, value):
        guid = value.get("guid")
        if guid is None:
            return None
        return self._cls(obj._guid_map, guid=guid)
    
    def to_json(self, value):
        return {"guid": value._guid}

    
def make_guid_property(wrapped):
    def __init__(self, label_guid):
        GuidProperty.__init__(self, label_guid, wrapped)

    return type(
        wrapped.__name__ + "Property",
        (GuidProperty,),
        {
            "__init__": __init__,
        }
    )

        
class Int64Property(AbstractSemanticGraphProperty):
    def from_json(self, obj, value):
        out = value.get("int64")
        if out is None:
            return None
        return int(out)

    def to_json(self, value):
        return {"int64": str(value)}
            
        
class StringProperty(AbstractSemanticGraphProperty):
    def from_json(self, obj, value):
        return value.get("string")

    def to_json(self, value):
        return {"string": value}

In [34]:
class Resource(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["coordinate"][0], guid_map, guid=guid)
    
    relative_path = StringProperty(names["resource.relative_path"][0])


ResourceProperty = make_guid_property(Resource)

In [35]:
class Coordinate(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["resource"][0], guid_map, guid=guid)
    
    x = Int64Property(names["coordinate.x"][0])
    y = Int64Property(names["coordinate.y"][0])


CoordinateProperty = make_guid_property(Coordinate)

In [36]:
class Rectangle(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(names["rectangle"][0], guid_map, guid=guid)
    
    anchor = CoordinateProperty(names["coordinate"][0])
    width = Int64Property(names["rectangle.width"][0])
    height = Int64Property(names["rectangle.height"][0])


RectangleProperty = make_guid_property(Rectangle)

In [37]:
class Bitmap(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(self, names["bitmap"][0], guid_map, guid=guid)
    
    resource = ResourceProperty(names["bitmap.resource"][0])
    rectangle = RectangleProperty(names["bitmap.rectangle"][0])
    
    
BitmapProperty = make_guid_property(Bitmap)

In [38]:
class Frame(SemanticGraphNode):
    def __init__(self, guid_map, guid=None):
        super().__init__(self, names["frame"][0], guid_map, guid=guid)
    
    duration_microseconds = Int64Property(names["frame.duration_microseconds"][0])
    bitmap = ResourceProperty(names["frame.bitmap"][0])


FrameProperty = make_guid_property(Frame)

In [39]:
resource = Resource(guid_map)
resource.relative_path = "./gamekitty.png"
resource

{"aba6ac79fd3d409da860a77c90942852": "bfa8113cb5e5436ebd76ab5418b7efd1", "4e09f9df1fdf4eb4964ff9ed1b375dbb": {"string": "./gamekitty.png"}}